## Introduction

Hello everyone, this kernel will serve as a starting point for many coming from the Coursera course "How to Win a Data Science Competition".
As many of you know, this competition "Predict Future Sales" by Russian software firm 1C Company is part of the final project for the course. 

The challenge here is to predict the total monthly sales of a product in each individual shop of a store chain, specifically November 2015. We are given the historical daily sales data from January 2013 to October 2015 in the provided file 'sales_train.csv'. From the training data we are to predict the sales numbers using the data given in 'test.csv', and create a submission file in the same format as 'sample_submission.csv'. Three other csv files are provided to give more insight into the data in the training set.

This kernel serves as the process for one of the exercises given in the course - to simply create a prediction for November 2015 using the exact historical sales data of the previous month, October 2015. The steps taken throughout this kernel may not be the most efficient, but it was my personal workflow of tackling the competition as a beginner data scientist without looking at the other kernels. 

In the future I should do visualizations for my EDA process, but I made some assumptions that the data is relatively clean, haha.

Please feel free to use this kernel as you like.

We will begin by importing the necessary libraries and the datasets.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("./"))

['sample_submission.csv', '.ipynb_checkpoints', 'shops.csv', 'test.csv', 'items.csv', 'Untitled.ipynb', 'item_categories.csv', 'sales_train.csv', '01_EDA.ipynb', 'previous-value-benchmark-simple-eda.ipynb']


In [2]:
train = pd.read_csv('sales_train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')
items = pd.read_csv('items.csv')
item_cats = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')

### The Benchmark - October 2015 historical sales to predict November 2015
Let's try to create a predictions benchmark by creating a submissions file with the previous month's sales. 

We will be using October 2015's sales data to predict November 2015's sales, and using the score as a benchmark for the evaluation of our future models.

In [3]:
train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [4]:
test.describe()

,ID,shop_id,item_id
count,214200.000000,214200.000000,214200.000000
mean,107099.500000,31.642857,11019.398627
std,61834.358168,17.561933,6252.644590
min,0.000000,2.000000,30.000000
25%,53549.750000,16.000000,5381.500000
50%,107099.500000,34.500000,11203.000000
75%,160649.250000,47.000000,16071.500000
max,214199.000000,59.000000,22167.000000


In [5]:
submission.describe()

,ID,item_cnt_month
count,214200.000000,214200.0
mean,107099.500000,0.5
std,61834.358168,0.0
min,0.000000,0.5
25%,53549.750000,0.5
50%,107099.500000,0.5
75%,160649.250000,0.5
max,214199.000000,0.5


Seems like the training set and testing set consist of different columns. 

The test set contains only the shop_id and item_id from the training set, and the submission file only contains the monthly item count with ID.

We need to map the training set shop_id and item_id to the ID number in the test set, add up the daily item counts for just October 2015, and create a similar submission file with ID and item_cnt_month.

In [6]:
train.head(50).T

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
date,02.01.2013,03.01.2013,05.01.2013,06.01.2013,15.01.2013,10.01.2013,02.01.2013,04.01.2013,11.01.2013,03.01.2013,...,30.01.2013,05.01.2013,02.01.2013,02.01.2013,19.01.2013,06.01.2013,07.01.2013,10.01.2013,12.01.2013,20.01.2013
date_block_num,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
shop_id,59,25,25,25,25,25,25,25,25,25,...,25,25,25,25,25,25,25,25,25,25
item_id,22154,2552,2552,2554,2555,2564,2565,2572,2572,2573,...,2720,2473,2480,2731,2731,2746,2746,2746,2746,2746
item_price,999,899,899,1709.05,1099,349,549,239,299,299,...,1699,899,58,599,599,299,299,299,299,299
item_cnt_day,1,1,-1,1,1,1,1,1,1,3,...,1,1,1,1,1,1,1,2,1,1


In [7]:
train['date'].describe()

count        2935849
unique          1034
top       28.12.2013
freq            9434
Name: date, dtype: object

In [8]:
'''
train['date'] = pd.to_datetime(train.date)
train = train.sort_values(by='date')
'''

"\ntrain['date'] = pd.to_datetime(train.date)\ntrain = train.sort_values(by='date')\n"

I tried to convert the 'date' column in the training set to datetime objects so I could sort it and split the October data, but it took way too long because there are 2.9 million rows in the data set. 

Luckily I noticed there is a date_block_num column that corresponds to the consecutive month of the dataset. 

In [9]:
train.tail(50).T

,2935799,2935800,2935801,2935802,2935803,2935804,2935805,2935806,2935807,2935808,...,2935839,2935840,2935841,2935842,2935843,2935844,2935845,2935846,2935847,2935848
date,13.10.2015,08.10.2015,29.10.2015,19.10.2015,22.10.2015,29.10.2015,19.10.2015,11.10.2015,11.10.2015,19.10.2015,...,24.10.2015,31.10.2015,11.10.2015,10.10.2015,09.10.2015,10.10.2015,09.10.2015,14.10.2015,22.10.2015,03.10.2015
date_block_num,33,33,33,33,33,33,33,33,33,33,...,33,33,33,33,33,33,33,33,33,33
shop_id,25,25,25,25,25,25,25,25,25,25,...,25,25,25,25,25,25,25,25,25,25
item_id,7524,7529,7531,7583,7590,7610,7612,7615,7624,7627,...,7315,7409,7393,7384,7409,7409,7460,7459,7440,7460
item_price,299,249,199,2150,1625,2890,250,250,250,349,...,399,299,349,749,299,299,299,349,299,299
item_cnt_day,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [10]:
train_oct2015 = train.loc[train['date_block_num'] == 33]

In [11]:
train_oct2015.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2882335,23.10.2015,33,45,13315,649.0,1.0
2882336,05.10.2015,33,45,13880,229.0,1.0
2882337,02.10.2015,33,45,13881,659.0,1.0
2882338,12.10.2015,33,45,13881,659.0,1.0
2882339,04.10.2015,33,45,13923,169.0,1.0


I've made a dataframe of just the October 2015 sales data. 

Now I will tally up a total item_cnt_month number for each unique shop_id-item_id pair.

In [25]:
df_m = train_oct2015.groupby(["shop_id", "item_id"])
month_sum = df_m.aggregate({"item_cnt_day":np.sum}).fillna(0)
month_sum.reset_index(level=["shop_id", "item_id"], inplace=True)
month_sum = month_sum.rename(columns={ month_sum.columns[2]: "item_cnt_month" })
month_sum.describe()

,shop_id,item_id,item_cnt_month
count,31531.000000,31531.000000,31531.000000
mean,31.628556,10896.730709,2.253528
std,16.491280,6165.331848,14.533295
min,2.000000,30.000000,-1.000000
25%,19.000000,5240.000000,1.000000
50%,31.000000,11056.000000,1.000000
75%,46.000000,15805.000000,2.000000
max,59.000000,22167.000000,2253.000000


In [26]:
submission.describe()

,ID,item_cnt_month
count,214200.000000,214200.0
mean,107099.500000,0.5
std,61834.358168,0.0
min,0.000000,0.5
25%,53549.750000,0.5
50%,107099.500000,0.5
75%,160649.250000,0.5
max,214199.000000,0.5


Something doesn't seem right - the number of rows is far less than the submission dataframe.

In [27]:
month_sum['item_id'].value_counts()

22102    42
3350     42
15069    42
10200    41
7223     41
         ..
8138      1
20686     1
1929      1
16604     1
14289     1
Name: item_id, Length: 5413, dtype: int64

In [28]:
test['item_id'].value_counts()

2047     42
19744    42
15406    42
13359    42
3240     42
         ..
21086    42
21214    42
17244    42
21342    42
2049     42
Name: item_id, Length: 5100, dtype: int64

I see what's going on. If the total item_cnt_month is 0 then item_id for the corresponding shop_id is not included. 

This could be fixed by simply merging month_sum with the test dataframe and filling the NaNs.

We will map the shop_id-item_id to ID number in the test set for our next step, and finally make a submission dataframe.

In [29]:
new_submission = pd.merge(month_sum, test, how='right', left_on=['shop_id','item_id'], right_on = ['shop_id','item_id']).fillna(0)
new_submission.drop(['shop_id', 'item_id'], axis=1)
new_submission = new_submission[['ID','item_cnt_month']]

The current score is quite horrible, at 8.53027. Let's try to clip the values within [0,20] as per the tip from the course.

In [30]:
new_submission['item_cnt_month'] = new_submission['item_cnt_month'].clip(0,20)
new_submission.describe()

,ID,item_cnt_month
count,214200.000000,214200.000000
mean,107099.500000,0.255649
std,61834.358168,1.089856
min,0.000000,0.000000
25%,53549.750000,0.000000
50%,107099.500000,0.000000
75%,160649.250000,0.000000
max,214199.000000,20.000000


In [18]:
new_submission.to_csv('previous_value_benchmark.csv', index=False)

The score now is 1.16777 as expected! Awesome. Now we can start trying some models in another kernel. 